# Анализ отзывов для улучшения модели предсказания рейтингов

## 1. Загрузка данных


### *1.1 Датасет состоит из 17 признаков :*

- hotel_address — адрес отеля;
- review_date — дата, когда рецензент разместил соответствующий отзыв;
- average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
- hotel_name — название отеля;
- reviewer_nationality — страна рецензента;
- negative_review — отрицательный отзыв, который рецензент дал отелю;
- review_total_negative_word_counts — общее количество слов в отрицательном отзыв;
- positive_review — положительный отзыв, который рецензент дал отелю;
- review_total_positive_word_counts — общее количество слов в положительном отзыве.
- reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;
- total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;
- total_number_of_reviews — общее количество действительных отзывов об отеле;
- tags — теги, которые рецензент дал отелю;
- days_since_review — количество дней между датой проверки и датой очистки;
- additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
- lat — географическая широта отеля;
- lng — географическая долгота отеля.

### *1.2 Импортируем необходимые библиотеки*

In [130]:

# Essential DS libraries
import numpy as np
import pandas as pd

# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
import torch


### *1.3 Загружаем данные*

In [131]:
hotels = pd.read_csv('hotels.csv')
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


## 2. Очистка данных

In [132]:
 # проводим быстрый разведывательный анализ 
import dtale
d= dtale.show(hotels)
#d.open_browser()

In [133]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

        - 2  признака имеют пропущенные значения: lat и lng- долгота и широта отеля 

### *2.2 Удаляем неинформативные признаки*

In [ ]:
# проверяем данные на неинформативные признаки
hotels.describe(include='object')

In [ ]:
hotels.describe()

,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,lat,lng
count,386803.000000,386803.000000,386803.000000,386803.000000,386803.000000,386803.00000,386803.000000,384355.000000,384355.000000
mean,498.246536,8.397231,18.538988,2743.992042,17.776985,7.17725,8.396906,49.443522,2.823402
std,500.258012,0.547881,29.703369,2316.457018,21.726141,11.05442,1.636090,3.466936,4.579043
min,1.000000,5.200000,0.000000,43.000000,0.000000,1.00000,2.500000,41.328376,-0.369758
25%,169.000000,8.100000,2.000000,1161.000000,5.000000,1.00000,7.500000,48.214662,-0.143649
50%,342.000000,8.400000,9.000000,2134.000000,11.000000,3.00000,8.800000,51.499981,-0.000250
75%,660.000000,8.800000,23.000000,3613.000000,22.000000,8.00000,9.600000,51.516288,4.834443
max,2682.000000,9.800000,408.000000,16670.000000,395.000000,355.00000,10.000000,52.400181,16.429233


План:

-  **hotel_address**: извлечь страну (последнее слово) и, возможно, город. Закодировать страну (и город, если решим оставить) одним из методов (one-hot, label encoding и т.д.).
Также, если есть возможность, можно добавить признаки расстояния до центра, но для этого нужны дополнительные данные. Пока из адреса извлечем только страну и город.

- **additional_number_of_scoring** - оставим как есть, но проведем анализ (например, корреляцию с целевой переменной).

- **review_date** - извлечем год и месяц.

- **average_score** - этот признак может быть полезен, но нужно быть осторожным, чтобы не было утечки. Поскольку мы предсказываем оценку конкретного отзыва, а average_score - это среднее по отелю, то это может быть полезным признаком (характеристика отеля). Однако, если average_score вычисляется на основе всех отзывов, включая текущий, то это утечка. Но из описания: "средний балл отеля, рассчитанный на основе последнего комментария за последний год" - не совсем ясно. Чтобы избежать утечки, лучше не использовать этот признак, либо убедиться, что он вычислен без учета текущего отзыва. В данном случае, поскольку описание неясное, мы можем его использовать, но будем помнить о риске утечки.

- **hotel_name** - удалим, так как это уникальный идентификатор, который может привести к переобучению.

- **reviewer_nationality** - можно сгруппировать редко встречающиеся национальности в одну категорию "Other", либо использовать частотное кодирование. Также можно попробовать оставить только топ-N национальностей.

- **negative_review** и **positive_review** - создадим бинарные признаки: есть отзыв или нет (1 - если отзыв не 'No Negative'/'No Positive', 0 - иначе). Также сами тексты отзывов можно обработать и извлечь из них признаки (например, тональность, длину и т.д.), но пока по плану только бинарный признак.

- **review_total_negative_word_counts** и **review_total_positive_word_counts** - удалим, так как они избыточны (длину можно получить из текста, а также мы уже создаем бинарные признаки наличия отзыва).

- **total_number_of_reviews** - оставим, это может быть полезно (популярность отеля).

- **total_number_of_reviews_reviewer_has_given** - оставим, как указано, может помочь в фильтрации неактивных reviewers.

- **reviewer_score** - это целевая переменная.

- **tags** - извлечем информацию, например, тип поездки (отпуск, бизнес и т.д.), состав группы (пара, семья и т.д.), длительность пребывания. Нужно разобрать строку и создать отдельные бинарные признаки.

- **days_since_review** - извлечем число из строки (например, "3 days" -> 3). Но стоит подумать, нужен ли этот признак. Если мы моделируем текущий рейтинг отеля, то давность отзыва может быть не важна, но если мы хотим учесть, что со временем качество услуг могло измениться, то можно оставить. Пока извлечем число.

- **lat** и **lng** - удалим, так как у нас уже есть адрес, и мы извлекли страну и город. Однако, если бы мы хотели использовать точное местоположение, то можно было бы оставить, но тогда нужно быть осторожным из-за возможного переобучения. Также можно было бы использовать для вычисления расстояний, но без конкретных точек интереса (центр, достопримечательности) это сложно. Удаляем.

Дополнительно:

Проверим на пропуски и дубликаты.

Возможно, потребуется стандартизация числовых признаков.

Количество уникальных тегов в датасете hotels: 0
Всего тегов (с повторениями): 0
Количество отзывов с тегами: 0

Топ-15 самых популярных тегов:


**Проверяем наличие дубликатов**


In [ ]:
# проверяем наличие дубликатов
print('Количество дубликатов: {}'.format(hotels[hotels.duplicated()].shape[0]))

Количество дубликатов: 307


In [ ]:
# удаляем дубликаты 
hotels.drop_duplicates()
hotels.shape        

(386803, 17)

In [ ]:
df= hotels.copy()
import pandas as pd
import ast


# Быстрый подсчет без создания новых колонок
all_tags = set()
for tags_str in df['tags'].dropna():
    try:
        tags = ast.literal_eval(tags_str)
        all_tags.update([tag.strip() for tag in tags])
    except:
        continue

print(f"Количество уникальных тегов: {len(all_tags)}")




Количество уникальных тегов: 2368


In [ ]:
import pandas as pd
import ast
from collections import Counter

# Загрузка датасета

# Функция для парсинга тегов
def parse_tags(tags_str):
    try:
        tags_list = ast.literal_eval(tags_str)
        return [tag.strip() for tag in tags_list]
    except:
        return []

# Применяем функцию к колонке с тегами
df['parsed_tags'] = df['tags'].apply(parse_tags)

# Создаем список всех тегов (для общего подсчета)
all_tags = []
for tags_list in df['parsed_tags']:
    all_tags.extend(tags_list)

# Считаем уникальные теги
unique_tags_count = len(set(all_tags))
print(f"Общее количество уникальных тегов: {unique_tags_count}")

# Считаем, в скольких отзывах встречается каждый тег
tag_review_count = Counter()

for tags_list in df['parsed_tags']:
    # Для каждого отзыва добавляем каждый уникальный тег один раз
    unique_tags_in_review = set(tags_list)
    for tag in unique_tags_in_review:
        tag_review_count[tag] += 1

# Находим самый популярный тег
most_common_tag, review_count = tag_review_count.most_common(1)[0]

print(f"\nСамый популярный тег: '{most_common_tag}'")
print(f"Он встречается в {review_count} отзывах")
print(f"Это {review_count/len(df)*100:.1f}% всех отзывов")

# Топ-10 самых популярных тегов
print(f"\nТоп-10 самых популярных тегов (по количеству отзывов):")
print("=" * 60)
for i, (tag, count) in enumerate(tag_review_count.most_common(10), 1):
    percentage = count / len(df) * 100
    print(f"{i:2d}. {tag:<50} {count:>5} отзывов ({percentage:.1f}%)")

Общее количество уникальных тегов: 2368

Самый популярный тег: 'Leisure trip'
Он встречается в 313593 отзывах
Это 81.1% всех отзывов

Топ-10 самых популярных тегов (по количеству отзывов):
 1. Leisure trip                                       313593 отзывов (81.1%)
 2. Submitted from a mobile device                     230778 отзывов (59.7%)
 3. Couple                                             189212 отзывов (48.9%)
 4. Stayed 1 night                                     145373 отзывов (37.6%)
 5. Stayed 2 nights                                    100263 отзывов (25.9%)
 6. Solo traveler                                      81235 отзывов (21.0%)
 7. Stayed 3 nights                                    72000 отзывов (18.6%)
 8. Business trip                                      61989 отзывов (16.0%)
 9. Group                                              49088 отзывов (12.7%)
10. Family with young children                         45836 отзывов (11.8%)


**Проверяем наличие неинформативных признаков**


In [ ]:
import pandas as pd
import ast
import re
from collections import Counter

# Загрузка данных
df = pd.read_csv('hotels.csv')

# Функция для парсинга тегов
def parse_tags(tags_str):
    try:
        tags_list = ast.literal_eval(tags_str)
        return [tag.strip() for tag in tags_list]
    except:
        return []

# Применяем функцию к колонке с тегами
df['parsed_tags'] = df['tags'].apply(parse_tags)

# Функция для извлечения количества ночей из тега
def extract_nights(tag):
    # Ищем шаблоны с количеством ночей
    patterns = [
        r'(\d+)\s*night',  # "2 nights", "3 night"
        r'Stayed\s*(\d+)\s*night',  # "Stayed 2 nights"
    ]
    
    for pattern in patterns:
        match = re.search(pattern, tag, re.IGNORECASE)
        if match:
            return int(match.group(1))
    return None

# Собираем информацию о количестве ночей
nights_data = []

for tags_list in df['parsed_tags']:
    for tag in tags_list:
        nights = extract_nights(tag)
        if nights is not None:
            nights_data.append(nights)

# Анализируем данные
if nights_data:
    nights_counter = Counter(nights_data)
    
    print("АНАЛИЗ ПРОДОЛЖИТЕЛЬНОСТИ ПРЕБЫВАНИЯ")
    print("=" * 50)
    
    # Самый популярный вариант
    most_common_nights, count = nights_counter.most_common(1)[0]
    print(f"Чаще всего останавливаются на: {most_common_nights} ночь(ей)")
    print(f"Количество таких случаев: {count}")
    print(f"Процент от всех упоминаний: {count/len(nights_data)*100:.1f}%")
    
    # Полная статистика
    print(f"\nРаспределение по количеству ночей:")
    print("-" * 30)
    total_mentions = len(nights_data)
    
    for nights, count in sorted(nights_counter.items()):
        percentage = count / total_mentions * 100
        print(f"{nights:2d} ночей: {count:4d} раз ({percentage:5.1f}%)")
    
    # Дополнительная статистика
    print(f"\nДополнительная статистика:")
    print(f"Всего упоминаний продолжительности: {total_mentions}")
    print(f"Средняя продолжительность: {sum(nights_data)/len(nights_data):.1f} ночей")
    print(f"Медианная продолжительность: {sorted(nights_data)[len(nights_data)//2]} ночей")
    print(f"Минимальная продолжительность: {min(nights_data)} ночей")
    print(f"Максимальная продолжительность: {max(nights_data)} ночей")
    
    # Визуализация топ-5
    print(f"\nТоп-5 самых частых продолжительностей:")
    print("-" * 40)
    for i, (nights, count) in enumerate(nights_counter.most_common(5), 1):
        percentage = count / total_mentions * 100
        print(f"{i}. {nights} ночей - {count} раз ({percentage:.1f}%)")
        
else:
    print("Не найдено тегов с информацией о количестве ночей")

АНАЛИЗ ПРОДОЛЖИТЕЛЬНОСТИ ПРЕБЫВАНИЯ
Чаще всего останавливаются на: 1 ночь(ей)
Количество таких случаев: 145373
Процент от всех упоминаний: 37.6%

Распределение по количеству ночей:
------------------------------
 1 ночей: 145373 раз ( 37.6%)
 2 ночей: 100263 раз ( 25.9%)
 3 ночей: 72006 раз ( 18.6%)
 4 ночей: 35748 раз (  9.2%)
 5 ночей: 15611 раз (  4.0%)
 6 ночей: 7399 раз (  1.9%)
 7 ночей: 5549 раз (  1.4%)
 8 ночей: 1910 раз (  0.5%)
 9 ночей:  966 раз (  0.2%)
10 ночей:  663 раз (  0.2%)
11 ночей:  306 раз (  0.1%)
12 ночей:  217 раз (  0.1%)
13 ночей:  174 раз (  0.0%)
14 ночей:  184 раз (  0.0%)
15 ночей:   87 раз (  0.0%)
16 ночей:   38 раз (  0.0%)
17 ночей:   27 раз (  0.0%)
18 ночей:   24 раз (  0.0%)
19 ночей:   23 раз (  0.0%)
20 ночей:   17 раз (  0.0%)
21 ночей:   19 раз (  0.0%)
22 ночей:    8 раз (  0.0%)
23 ночей:    6 раз (  0.0%)
24 ночей:    5 раз (  0.0%)
25 ночей:    4 раз (  0.0%)
26 ночей:    6 раз (  0.0%)
27 ночей:   10 раз (  0.0%)
28 ночей:    7 раз (  0.0

d:\idi\.venv\lib\site-packages\dtale\app.py:445: FutureWarning:

`torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead



In [ ]:
#проверяем на неиформативные признаки
low_information_cols= []

#цикл по всем столбцам
for col in hotels.columns:
    #наибольшая относительная частота 
    top_freq= hotels[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio= hotels[col].nunique() / hotels[col].count()
    #сравниваем наибольшую частоту с порогом
    if top_freq> 0.90:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100)}% одинаковых значений')
    if nunique_ratio>0.90:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100)}% уникальных значений')        
        

In [ ]:
# удаляем неинформативные признаки 
hotels= hotels.drop(['hotel_name', 'review_total_negative_word_counts' ,'review_total_positive_word_counts', 'lat', 'lng'], axis=1)
print(f'Результирующее число признаков: {hotels.shape[1]}')

Результирующее число признаков: 12


## обучение модели 

In [ ]:
# производим очистку данных  для теста
# # убираем признаки которые еще не успели обработать, 
# # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
#object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
#hotels.drop(object_columns, axis = 1, inplace=True)

# # заполняем пропуски самым простым способом
#hotels = hotels.fillna(0)

In [ ]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
   
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
  
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)

ValueError: could not convert string to float: 'Pla a de Llevant s n Sant Mart 08019 Barcelona Spain'

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.1413411982212641
